In [ ]:
# download monthly WOA18 0.25 deg salinity files

! PYTHONPATH=../../pycommon ./get_monthly_woa18.py woa_test \
--start_month 1 --end_month 12 \
--res 0.25 \
--dvar salinity 

In [ ]:
# generate interpolating weights from latlon grids of WOA18 to MOM6 tripolar grids

! pwd
! ./gen_regridder_woa18.py wts.nc \
--target_grid_path ../../test_data/flood_salt_restore_PHC2.1440x1080.v20180405.nc \
--source_grid_path ../../test_data/woa18_decav_s01_04.nc

In [ ]:
# create 12-month WOA18 latlon SSS by aggregating monthly SSS.
# it will generate two files, one file with land unfilled, and the other with land filled.

! PYTHONPATH=../../pycommon ./fill_sss_woa18_12months.py ../../test_data/woa18 \
--start_month 1 --end_month 12

In [ ]:
# Intepolate latlon 12-month SSS to tripolar grids

! ./remap_sss_latlon2tripolar_woa18.py template.1440x1080.woa18.nc \
        --regridder_file_path wts.nc \
        --wet_grid_path ../../test_data/ocean_topog.nc \
        --target_grid_path ../../test_data/flood_salt_restore_PHC2.1440x1080.v20180405.nc \
        --source_grid_path ../../test_data/woa18_decav_s01_04.nc \
        --sss_filled_path filled_sss_s01_e12.nc_backup

In [ ]:
# Two Checks: 

%matplotlib inline

import numpy as np
from netCDF4 import Dataset

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
import sys
sys.path.append("../../pycommon")
from pps_tools import set_cartopy_gridlines, set_cartopy_colorbar


# unfilled one
f = Dataset('unfilled_sss_s01_e12.nc')
sss_unfilled = f.variables['unfilled'][:]
lat1d = f.variables['lat'][:]
lon1d = f.variables['lon'][:]
f.close()

# filled one
f = Dataset('filled_sss_s01_e12.nc')
sss_filled = f.variables['unfilled'][:]
lat1d = f.variables['lat'][:]
lon1d = f.variables['lon'][:]
f.close()

In [ ]:
# Check: 12-month SSS with its land value missing

fig, axs = plt.subplots(nrows=1,ncols=2,
                        subplot_kw={'projection': ccrs.Robinson()},
                        figsize=(11,8.5))
crs = ccrs.PlateCarree()

for i in range(2):
    axs[i].coastlines(linewidth=0.75, color='black', resolution='50m')
    axs[i].gridlines(crs=crs, linestyle='-')
    #axs[i].set_extent([-150, -30, 0, 30], crs=crs)
    if i==0:
        month = 6
    elif i==1:
        month = 7
    #surf = axs[i].contourf(lon1d, lat1d, sss_unfilled[month-1,:,:], levels=np.arange(32,38.5,0.5), extend="both", cmap=plt.cm.jet, transform=crs)
    surf = axs[i].pcolormesh(lon1d, lat1d, sss_unfilled.data[month-1,:,:], vmin=32, vmax=38.5, edgecolors=None, cmap=plt.cm.jet, transform=crs)

    set_cartopy_colorbar(axs[i],surf,fig,shrink=0.3)

    axs[i].add_feature(cfeature.COASTLINE)
    axs[i].add_feature(cfeature.LAND, facecolor=[0.7,0.7,0.7])
    set_cartopy_gridlines(ax=axs[i],crs=crs,ytick=np.arange(-90,90+10,30))
    axs[i].title.set_text("WOA18 SSS on lat-lon in NH in Month {}".format(month))


In [ ]:
# Check: 12-month SSS with its land value filled


fig, axs = plt.subplots(nrows=1,ncols=2,
                        subplot_kw={'projection': ccrs.Robinson()},
                        figsize=(11,8.5))
crs = ccrs.PlateCarree()

for i in range(2):
    axs[i].coastlines(linewidth=0.75, color='black', resolution='50m')
    axs[i].gridlines(crs=crs, linestyle='-')
    #axs[i].set_extent([-150, -30, 0, 30], crs=crs)
    if i==0:
        month = 6
    elif i==1:
        month = 7
    #surf = axs[i].contourf(lon1d, lat1d, sss_unfilled[month-1,:,:], levels=np.arange(32,38.5,0.5), extend="both", cmap=plt.cm.jet, transform=crs)
    surf = axs[i].pcolormesh(lon1d, lat1d, sss_filled.data[month-1,:,:], vmin=32, vmax=38.5, edgecolors=None, cmap=plt.cm.jet, transform=crs)

    set_cartopy_colorbar(axs[i],surf,fig,shrink=0.3)

    axs[i].add_feature(cfeature.COASTLINE)
    axs[i].add_feature(cfeature.LAND, facecolor=[0.7,0.7,0.7])
    set_cartopy_gridlines(ax=axs[i],crs=crs,ytick=np.arange(-90,90+10,30))
    axs[i].title.set_text("WOA18 SSS on lat-lon in NH in Month {}".format(month))



In [ ]:
# Check if SSS file is updated.


f = Dataset('template.1440x1080.woa18.nc')
lon2d_tri = f.variables['lon'][:]
lat2d_tri = f.variables['lat'][:]
sss2d_updated_tri = f.variables['SALT'][:]
f.close()

f = Dataset('../../test_data/flood_salt_restore_PHC2.1440x1080.v20180405.nc')
lon2d_tri = f.variables['lon'][:]
lat2d_tri = f.variables['lat'][:]
sss2d_tri = f.variables['SALT'][:]
f.close()

fig, axs = plt.subplots(nrows=1,ncols=2,
                        subplot_kw={'projection': ccrs.Robinson()},
                        figsize=(11,8.5))
crs = ccrs.PlateCarree()

month = 7 
for i in range(2):
    axs[i].coastlines(linewidth=0.75, color='black', resolution='50m')
    axs[i].gridlines(crs=crs, linestyle='-')
    #axs[i].set_extent([-150, -30, 0, 30], crs=crs)
    if i==0:
        surf = axs[i].pcolormesh(lon2d_tri, lat2d_tri, sss2d_tri[month-1,:,:], vmin=32, vmax=38.5, edgecolors=None, cmap=plt.cm.jet, transform=crs)
        title_str = "original"
    elif i==1:
        surf = axs[i].pcolormesh(lon2d_tri, lat2d_tri, sss2d_updated_tri[month-1,:,:], vmin=32, vmax=38.5, edgecolors=None, cmap=plt.cm.jet, transform=crs)
        title_str = "updated"
        
    set_cartopy_colorbar(axs[i],surf,fig,shrink=0.3)

    axs[i].add_feature(cfeature.COASTLINE)
    axs[i].add_feature(cfeature.LAND, facecolor=[0.7,0.7,0.7])
    set_cartopy_gridlines(ax=axs[i],crs=crs,ytick=np.arange(-90,90+10,30))
    axs[i].title.set_text(title_str)


